In [1]:
email_conversation = """From: John Smith john.smith@example.comTo: Emily Johnson emily.johnson@example.comDate: February 3, 2025

Dear Emily,

I hope this email finds you well. I am reaching out to explore the possibility of a collaboration between our companies. We have been following your recent projects, and we believe that working together could bring mutual benefits.

Would you be available for a quick call next week to discuss this further? Please let me know a time that works for you.

Looking forward to your response.

Best regards,John SmithXYZ Solutions"""

In [2]:
from pydantic import BaseModel, Field


# 이메일 본문으로부터 주요 엔티티 추출
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    company: str = Field(description="메일을 보낸 사람의 회사 정보")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [3]:
## LCEL 구조

# chain = prompt | llm | output_parser

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [5]:
from langchain_core.output_parsers import PydanticOutputParser

# PydanticOutputParser 생성
output_parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

#QUESTION:
다음의 이메일 내용 중에서 주요 내용을 추출해 주세요.

#EMAIL CONVERSATION:
{email_conversation}

#FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=output_parser.get_format_instructions())

In [7]:
# 체인 생성
chain = prompt | llm | output_parser

In [8]:
# 체인 실행
answer = chain.invoke({"email_conversation": email_conversation})

In [9]:
print(answer.summary)

John Smith는 Emily Johnson에게 두 회사 간의 협업 가능성을 논의하기 위해 연락을 취했습니다. 그는 다음 주에 전화 통화를 통해 더 자세히 논의하고 싶다고 제안했습니다.


In [10]:
answer

EmailSummary(person='John Smith', company='XYZ Solutions', email='john.smith@example.com', subject='협업 가능성 논의', summary='John Smith는 Emily Johnson에게 두 회사 간의 협업 가능성을 논의하기 위해 연락을 취했습니다. 그는 다음 주에 전화 통화를 통해 더 자세히 논의하고 싶다고 제안했습니다.', date='다음 주')

## 검색: SERP API

참고: https://serpapi.com/integrations/python

In [12]:
import os

os.environ["SERPAPI_API_KEY"] = "user_api_key"

In [13]:
from langchain_community.utilities import SerpAPIWrapper

params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "3"}

search = SerpAPIWrapper(params=params)

In [14]:
answer.email

'john.smith@example.com'

In [15]:
query = f"{answer.person} {answer.company} {answer.email}"
query

'John Smith XYZ Solutions john.smith@example.com'

In [16]:
search_result = search.run(query)

In [17]:
search_result = eval(search_result)

In [18]:
type(search_result)

list

In [19]:
# 검색 결과
search_result_string = "\n".join(search_result)

In [20]:
answer

EmailSummary(person='John Smith', company='XYZ Solutions', email='john.smith@example.com', subject='협업 가능성 논의', summary='John Smith는 Emily Johnson에게 두 회사 간의 협업 가능성을 논의하기 위해 연락을 취했습니다. 그는 다음 주에 전화 통화를 통해 더 자세히 논의하고 싶다고 제안했습니다.', date='다음 주')

In [21]:
from langchain_core.prompts import PromptTemplate

report_prompt = PromptTemplate.from_template(
    """당신은 이메일의 주요 정보를 바탕으로 요약 정리해 주는 전문가 입니다.
당신의 임무는 다음의 이메일 정보를 바탕으로 보고서 형식의 요약을 작성하는 것입니다.
주어진 정보를 기반으로 양식(format)에 맞추어 요약을 작성해 주세요.

#Information:
- Sender: {sender}
- Additional Information about sender: {additional_information}
- Company: {company}
- Email: {email}
- Subject: {subject}
- Summary: {summary}
- Date: {date}

#Format(in markdown format):
🙇‍♂️ 보낸 사람:
- (보낸 사람의 이름, 회사 정보)

📧 이메일 주소:
- (보낸 사람의 이메일 주소)

😍 보낸 사람과 관련하여 검색된 추가 정보:
- (검색된 추가 정보)

✅ 주요 내용:
- (이메일 제목, 요약)

⏰ 일정:
- (미팅 날짜 및 시간)

#Answer:"""
)

In [22]:
from langchain_core.output_parsers import StrOutputParser

In [23]:
report_chain = (
    report_prompt | ChatOpenAI(model="gpt-4-turbo", temperature=0) | StrOutputParser()
)

In [24]:
report_response = report_chain.invoke(
    {
        "sender": answer.person,
        "additional_information": search_result_string,
        "company": answer.company,
        "email": answer.email,
        "subject": answer.subject,
        "summary": answer.summary,
        "date": answer.date,
    }
)

In [25]:
print(report_response)

🙇‍♂️ 보낸 사람:
- John Smith, Marketing Director at XYZ Solutions

📧 이메일 주소:
- john.smith@example.com

😍 보낸 사람과 관련하여 검색된 추가 정보:
- John Smith는 XYZ Solutions의 마케팅 디렉터로, 샌프란시스코와 램지에 위치한 경험을 가지고 있습니다. LinkedIn 프로필을 통해 10억 명의 전문가 커뮤니티에 속해 있습니다.

✅ 주요 내용:
- 제목: 협업 가능성 논의
- 요약: John Smith는 Emily Johnson에게 두 회사 간의 협업 가능성을 논의하기 위해 연락을 취했습니다. 그는 다음 주에 전화 통화를 통해 더 자세히 논의하고 싶다고 제안했습니다.

⏰ 일정:
- 다음 주
